# Personality Aware Recommender System: MovieLens Replication and Experiment

## Setup
This first cell is for importing required libraries and required pip installs

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162996 sha256=ae1d00cfa65b50d6e73c1abf84d7e70cdddf25e99bb3c9f653f501a3afb98791
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [15]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate, KFold
from surprise import KNNBasic
from surprise import PredictionImpossible
import heapq
import numpy as np
import json

## Preprocessing/importing data

In [8]:
movies = pd.read_csv("aggregate_movie_data.csv")
personality = pd.read_csv("personality-data.csv")
ratings = pd.read_csv("updatedRatings.csv")
ratings.rename(columns={"useri": "userid", ' movie_id': 'movie_id', ' rating': 'rating'})

# Opening JSON files
f = open('result.json')
personalityAndRatingsDict = json.load(f)

f = open('userIdToPersonalityTraitDict.json')
userIdToPersonalityTraitDict = json.load(f)

f = open('movieDetails.json')
movieDetailsDict = json.load(f)

## Defining important variables

In [9]:
pTraits = ["openness", "conscientiousness", "extraversion", "agreeableness", "emotional_stability"]
levelOfTraits = ["lowTrait", "moderateTrait", "highTrait"]
ratingLevels = ["lowRating", "moderateRating", "highRating"]
popularityLevels = ["lowPopularity", "moderatePopularity" ,"highPopularity"]
genres = ["is_action", "is_adventure", "is_animation", "is_children", "is_comedy",
          "is_crime", "is_documentary", "is_drama", "is_fantasy", "is_horror", "is_musical",
          "is_mystery", "is_romance", "is_science_fiction", "is_thriller", "is_war", "is_western"]


# Personality Traits

In [11]:
# Logic to recreate table 2 from the table

import json

f = open('userIdToPersonalityTraitDict.json')
user_personalities = json.load(f)

num_low_open = 0
num_high_open = 0
num_low_conscientiousness = 0
num_high_conscientiousness = 0
num_low_extraversion = 0
num_high_extraversion = 0
num_low_agreeableness = 0
num_high_agreeableness = 0
num_low_emotional_stability = 0
num_high_emotional_stability = 0
for user in user_personalities:
  if user_personalities[user]['openness'] == 'lowTrait':
    num_low_open += 1
  if user_personalities[user]['openness'] == 'highTrait':
    num_high_open += 1
  if user_personalities[user]['conscientiousness'] == 'lowTrait':
    num_low_conscientiousness += 1
  if user_personalities[user]['conscientiousness'] == 'highTrait':
    num_high_conscientiousness += 1
  if user_personalities[user]['extraversion'] == 'lowTrait':
    num_low_extraversion += 1
  if user_personalities[user]['extraversion'] == 'highTrait':
    num_high_extraversion += 1
  if user_personalities[user]['agreeableness'] == 'lowTrait':
    num_low_agreeableness += 1
  if user_personalities[user]['agreeableness'] == 'highTrait':
    num_high_agreeableness += 1
  if user_personalities[user]['emotional_stability'] == 'lowTrait':
    num_low_emotional_stability += 1
  if user_personalities[user]['emotional_stability'] == 'highTrait':
    num_high_emotional_stability += 1

user_info = [["Openness", num_low_open, num_high_open], ["Concientiousness", num_low_conscientiousness, num_high_conscientiousness], ["Extraversion", num_low_extraversion, num_high_extraversion],
 ["Agreeablness", num_low_agreeableness, num_high_agreeableness], ["Emotinal Stability", num_low_emotional_stability, num_high_emotional_stability]]

personality_info_df = pd.DataFrame(user_info, columns=["Personality Trait", "# Low Users", "# High Users"])
print("Table 2")
personality_info_df

Table 2


,Personality Trait,# Low Users,# High Users
0,Openness,119,724
1,Concientiousness,57,411
2,Extraversion,431,147
3,Agreeablness,62,198
4,Emotinal Stability,100,426


## Standard User-User CF

In [10]:
# Define the format
reader = Reader(rating_scale=(ratings[' rating'].min(), ratings[' rating'].max()))

# Load the data from the dataframe
data = Dataset.load_from_df(ratings[['useri', ' movie_id', ' rating']], reader)

model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})

# Perform cross-validation and print results
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9129  0.9107  0.9133  0.9126  0.9162  0.9131  0.0018  
MAE (testset)     0.6846  0.6820  0.6838  0.6848  0.6868  0.6844  0.0015  
Fit time          11.74   13.60   13.79   15.20   14.48   13.76   1.16    
Test time         70.01   71.12   71.34   69.88   71.88   70.85   0.78    


# Weight Traits Uniformly
Helper function that returns an offset based on tendencies of the user's trait(s) and the movie's attributes

In [12]:
import json

f = open('movieDetails.json')
movie_details = json.load(f)

f = open('result.json')
avg_ratings = json.load(f)

# return the offset for the prediction by uniformly weighting the traits
def uniform_weights(user_id, movie_id):
  movie_id = str(float(movie_id))
  if movie_id not in movie_details:
    return 0

  # extract the movie's rating level, popularity level, and the list of genres
  rating_level = movie_details[movie_id]['ratingLevel']
  popularity_level = movie_details[movie_id]['popularityLevel']
  genres = list(movie_details[movie_id]['setOfGenres'].keys())

  # for a user that has multiple personality traits, we keep track of them here
  low_traits = []
  high_traits = []

  # check each personality trait and if it is high or low append it to its respective list
  if user_personalities[user_id]['openness'] == 'lowTrait':
    low_traits.append('openness')
  if user_personalities[user_id]['openness'] == 'highTrait':
    high_traits.append('openness')
  if user_personalities[user_id]['conscientiousness'] == 'lowTrait':
    low_traits.append('conscientiousness')
  if user_personalities[user_id]['conscientiousness'] == 'highTrait':
    high_traits.append('conscientiousness')
  if user_personalities[user_id]['extraversion'] == 'lowTrait':
    low_traits.append('extraversion')
  if user_personalities[user_id]['extraversion'] == 'highTrait':
    high_traits.append('extraversion')
  if user_personalities[user_id]['agreeableness'] == 'lowTrait':
    low_traits.append('agreeableness')
  if user_personalities[user_id]['agreeableness'] == 'highTrait':
    high_traits.append('agreeableness')
  if user_personalities[user_id]['emotional_stability'] == 'lowTrait':
    low_traits.append('emotional_stability')
  if user_personalities[user_id]['emotional_stability'] == 'highTrait':
    high_traits.append('emotional_stability')

  # list of the weights for each trait
  # low_weight[i] is the offset for the trait low_traits[i], same for high
  low_weights = []
  high_weights = []

  # find the offset for each low weight
  for low_trait in low_traits:

    # if a movie has multiple genres, we take the average
    total_diff = 0
    for genre in genres:
      moderate_rating = avg_ratings[low_trait]["moderateTrait"][rating_level][popularity_level][genre] # moderate rating for this type of movie for this trait
      low_trait_rating = avg_ratings[low_trait]["lowTrait"][rating_level][popularity_level][genre] # low rating for this type of movie for this trait
      total_diff += low_trait_rating - moderate_rating # difference of moderate and low, this number will be negative (decrease overall rating)
    if len(genres) > 0:
      low_weights.append(total_diff / len(genres))

  # find the offset for each high weight
  for high_trait in high_traits:

    # if a movie has multiple genres, we take the average
    total_diff = 0
    for genre in genres: # take into account each genre, take the average of all rating difference between moderate and high
      moderate_rating = avg_ratings[high_trait]["moderateTrait"][rating_level][popularity_level][genre]
      high_trait_rating = avg_ratings[high_trait]["highTrait"][rating_level][popularity_level][genre]
      total_diff += high_trait_rating - moderate_rating
    if len(genres) > 0:
      high_weights.append(total_diff / len(genres))

  # combine low and high trait offsets
  all_weights = low_weights + high_weights
  if len(all_weights) == 0:
    return 0

  # take average of all offsets for all the personalities this person has
  final_offset = np.mean(all_weights)
  return final_offset


In [13]:
# test offset to show functionality of helper function
# for this movie and user, the rating will be increased by 0.038
test_offset = uniform_weights("875afaef12d7872b644948db1bb3040a", 122)
print(test_offset)

0.03847020837482931


In [16]:
# Custom rmse function to just handle two lists, not Prediction objects. We do this because the estimates in prediction objects are not mutable, so we can't alter the predictions like we want
import math

def compute_rmse(predicted_ratings, actual_ratings):
    if len(predicted_ratings) != len(actual_ratings):
        raise ValueError("Length of predicted ratings and actual ratings must be the same.")

    # Compute the squared differences between predicted and actual ratings
    squared_diffs = [(pred - actual) ** 2 for pred, actual in zip(predicted_ratings, actual_ratings)]

    # Compute the mean squared difference
    mean_squared_diff = sum(squared_diffs) / len(predicted_ratings)

    # Compute the RMSE (square root of the mean squared difference)
    rmse = math.sqrt(mean_squared_diff)

    return rmse

In [17]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})

normal_rmse_scores = []
uniform_weight_rmse_scores = []
for trainset, testset in kf.split(data):

  # train and test algorithm.
  algo.fit(trainset)
  predictions = algo.test(testset)

  # create lists containing the actual and predicted rating from the prediction object for each prediction
  uniform_pred_list = []
  actual_list = []

  # keep track of how many predictions we altered
  total_predictions = 0
  modified_predictions = 0
  for i in range(len(predictions)):
    # offset with uniform weights
    uniform_offset = uniform_weights(predictions[i].uid, int(predictions[i].iid))
    if uniform_offset != 0:
      modified_predictions += 1
    new_prediction = predictions[i].est + uniform_offset
    uniform_pred_list.append(new_prediction)

    #actual predictions
    actual_list.append(predictions[i].r_ui)
  total_predictions += len(predictions)

  # Compute and print Root Mean Squared Error
  uniform_rmse = compute_rmse(uniform_pred_list, actual_list)
  uniform_weight_rmse_scores.append(uniform_rmse)

print('uniformly weighted average rmse:', np.mean(uniform_weight_rmse_scores))
print(f'{modified_predictions}/{total_predictions} predicitons modified')

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
uniformly weighted average rmse: 0.9109085421453411
158523/202211 predicitons modified


## Personality Aware User-User CF Model

The personality aware model is largely based off of the user-user predicted score. The user-user predicted score is then altered by two things, the average score (increased overall accuracy) and the average score of each trait to genre relationship. The average score of each trait to a specific genre was calculated by iterating through each trait and identifying the trait level (low, moderate, high) and identifying the movie's popularity and rating level. Once these things are found, we iterate through the genre's of the movie and find the average rating that users that match that specific trait and trait level have rated movies that match the popularity and rating level. Much of this is sped up with the preprocessed data as we are able to access a dictionary thats layout follows, dictionary[personality trait][trait level][rating level][popularity level][genre] and the final output of this dictionary at that index would be the average score.

In [ ]:
# Inherits from KNNBasic (User User)
class KNNBasicWithPersonality(KNNBasic):
  # Overriding estimate function
  # Input: u - userId
  #        i - itemId
  # Return: Prediced score and Details
  def estimate(self, u, i):
    # Establish trait names and weights
    Traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "emotional_stability"]
    if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
        raise PredictionImpossible("User and/or item is unknown.")

    x, y = self.switch(u, i)

    neighbors = [(self.sim[x, x2], r) for (x2, r) in self.yr[y]]
    k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[0])

    # compute weighted average
    sum_sim = sum_ratings = actual_k = 0
    for (sim, r) in k_neighbors:
        if sim > 0:
            sum_sim += sim
            sum_ratings += sim * r
            actual_k += 1

    if actual_k < self.min_k:
        raise PredictionImpossible("Not enough neighbors.")

    # Convert model Id's to raw Id's
    rawId = self.trainset.to_raw_uid(u)
    rawMovieId = self.trainset.to_raw_iid(i)
    rawMovieId = str(float(rawMovieId))

    # Prepare data
    personalityData = self.sim_options["personalityData"]
    personalityData = personalityData[personalityData["userid"] == rawId]
    movieData = self.sim_options["movieData"]
    personalityWeight = self.sim_options["personalityWeight"]

    # aggScore = Aggregate score added to standard user user predicted score
    aggScore = 0

    # If movieId is valid
    # if rawMovieId in movieData:
    movieData = movieData[rawMovieId]
    countOfTraits = 0
    if len(movieData["setOfGenres"]) > 0:
        personalityAndRatingsDict = self.sim_options["personalityAndRatingsDict"]
        userIdToPersonalityTraitDict = self.sim_options["userIdToPersonalityTraitDict"]
        movieRatingLevel = movieData["ratingLevel"]
        moviePopularityLevel = movieData["popularityLevel"]
        for trait in Traits:
            traitLevel = userIdToPersonalityTraitDict[rawId][trait]
            traitScore = 0
            for genre in movieData["setOfGenres"]:
                traitScore += personalityAndRatingsDict[trait][traitLevel][movieRatingLevel][moviePopularityLevel][genre]
            aggScore += traitScore / len(movieData["setOfGenres"])
            countOfTraits += 1
        aggScore /= 5

    est = (((1-personalityWeight)/2) * (sum_ratings / sum_sim)) + (((1-personalityWeight)/2) * movieData["average_score"]) + (personalityWeight * aggScore)

    details = {"actual_k": actual_k}

    return est, details

## Run Personality Aware Model - Cross Validate

In [ ]:
personalityWeight = 0.23224511718044732

# Define the format
reader = Reader(rating_scale=(ratings[' rating'].min(), ratings[' rating'].max()))
# Load the data from the dataframe
data = Dataset.load_from_df(ratings[['useri', ' movie_id', ' rating']], reader)
model = KNNBasicWithPersonality(sim_options={
  'name': 'cosine',
  'user_based': True,
  "personalityAndRatingsDict": personalityAndRatingsDict,
  "userIdToPersonalityTraitDict": userIdToPersonalityTraitDict,
  "personalityData": personality,
  "movieData": movieDetailsDict,
  "personalityWeight": personalityWeight,
  "genres": genres
})

results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print(results)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasicWithPersonality on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8885  0.8872  0.8926  0.8859  0.8882  0.8885  0.0023  
MAE (testset)     0.6841  0.6837  0.6872  0.6835  0.6843  0.6846  0.0014  
Fit time          4.44    4.52    4.50    4.51    4.47    4.49    0.03    
Test time         60.83   61.26   60.73   60.90   61.40   61.02   0.26    
{'test_rmse': array([0.88846538, 0.88723197, 0.89263614, 0.88588207, 0.88822942]), 'test_mae': array([0.68410402, 0.6837298 , 0.68723688, 0.68351404, 0.68427325]), 'fit_ti

## Gradient Descent

The goal of gradient descent is to find the optimized weight that personality traits should be taken into account compared to the normal user-user and the average score (both included in the personality aware model). The way we did this was to use this equation, predicted score = ((1-weight)/2) * user-user predicted score + ((1-weight)/2) * average score + (weight) * personality trait based score. By using gradient descent, we can cross validate the personality based recommender and alter the weight. Since the cross validation outputs an RMSE, we can just alter the weight depending on the gradient descent algorithm that is documented below.

In [ ]:
import numpy as np

# Define the format
reader = Reader(rating_scale=(ratings[' rating'].min(), ratings[' rating'].max()))
# Load the data from the dataframe
data = Dataset.load_from_df(ratings[['useri', ' movie_id', ' rating']], reader)

# Cost Function - Runs Cross validation that outputs the RMSE
def compute_rmse(weight):
  model = KNNBasicWithPersonality(sim_options={
    'name': 'cosine',
    'user_based': True,
    "personalityAndRatingsDict": personalityAndRatingsDict,
    "userIdToPersonalityTraitDict": userIdToPersonalityTraitDict,
    "personalityData": personality,
    "movieData": movieDetailsDict,
    "personalityWeight": weight,
    "genres": genres
  })
  results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
  rmse = np.mean(results['test_rmse'])
  return rmse

# Gradient Descent Function
def gradient_descent(init_weight, learning_rate, iterations):
  # Obtain Weight from initial weight
  weight = init_weight
  # Run N iterations
  h = .001  # Small step for numerical gradient
  for i in range(iterations):
    print("Starting step", i)
    # Get current Cost
    current_rmse = compute_rmse(weight)
    print(f"Iteration {i}, RMSE: {current_rmse}, Weight: {weight}")
    grad = 0
    # Adjust weight by h
    weight += h
    print(f"Testing with weight: {weight}")
    # Compute RSME with adjusted trait
    rmse_with_h = compute_rmse(weight)
    print(f"RMSE tested {rmse_with_h}")
    # Compute derivitive according to weight
    grad = (rmse_with_h - current_rmse) / h
    print(f"Gradient: {grad}")
    # Reset trait back to original weight
    weight -= h

    # After test, descend gradient
    print(f"Gradient changing by {learning_rate * grad}")
    weight -= learning_rate * grad
    print(f"New weight: {weight}")

  print("Final", weight)
  return weight

# Initialize weights
initial_weight = 0.23224511718044732

# Parameters for the gradient descent
learning_rate = .1
iterations = 20

# Run gradient descent
optimized_weights = gradient_descent(initial_weight, learning_rate, iterations)


Starting step 0
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Iteration 0, RMSE: 0.88862820442151, Weight: 0.23141492101871536
Testing with weight: 0.23241492101871536
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE tested 0.8886299024598927
Gradient: 0.0016980383826803447
Gradient changing by 0.00016980383826803447
New weight:

# Final Results
Normal User-User RMSE: 0.9131

User-User with Uniform Personality Weights: 0.9109

User-User with learned Weight via Gradient Descent: 0.8859

In conclusion, these results show that personality-aware recommender systems do slightly improve recommendation quality. Learning the weights via gradient descent took many hours of training, so implementing this in practice may not be worth it if it only offers an RMSE improvement of 0.03. To simply cross validate the personality aware model, it took around 2.5 times the amount of time compared to normal User-User. We believe that the importance of personality traits is nullified due to the nature of User-User, since the ratings are based on similar users, it can be assumed that the users are already similar to each other.

